By: Michał Marusiński

In [2]:
import pandas as pd
import numpy as np

# key_pic.xlsx - details regarding pictures
+ name - name of a picture from IAPS database (example: original picture - neg_IAPS_9181.png.png; high contrast - hi_neg_IAPS_9181.png; so there is suffix 'hi_' for high contrast pictures)
+ trigger - picture id used accross the whole study (example: original picture - 41211; high contrast: 41212; so original pictures ends with 1 and high contrast pictures ends with 2)
+ the rest of the columns is not important, for now

In [22]:
key_pic = pd.read_excel(r"C:\Users\48660\Desktop\Development Workshop\keys\key_pic.xlsx")
key_pic.head()
key_pic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               240 non-null    object
 1   trigger            240 non-null    int64 
 2   var2               240 non-null    int64 
 3   var1               240 non-null    int64 
 4   valence_category   240 non-null    int64 
 5   stim_number        240 non-null    int64 
 6   saliency_category  240 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 13.3+ KB


# participants_pic.xlx - information about participants
+ numer osoby - person id in the study
+ płeć - their gender
+ kolor oczu - eye colour
+ kalibracja_PICTURE1, kalibracja_PICTURE2, kalibracja_PICTURE3 - device calibration on 3 pictures
+ Dodatkowe uwagi - there are always some problems in such experiments...

In [4]:
#reading a file
participants_pic = pd.read_excel(r"C:\Users\48660\Desktop\Development Workshop\keys\participants_pic.xlsx")
participants_pic.head()

#splitting values in kalibracja_PICTUREX to separate columns
participants_pic[['kalibracja_PICTURE1_value_1', 'kalibracja_PICTURE1_value_2']] = participants_pic['kalibracja_PICTURE1'].str.split(';', expand=True)
participants_pic[['kalibracja_PICTURE2_value_1', 'kalibracja_PICTURE2_value_2']] = participants_pic['kalibracja_PICTURE2'].str.split(';', expand=True)
participants_pic[['kalibracja_PICTURE3_value_1', 'kalibracja_PICTURE3_value_2']] = participants_pic['kalibracja_PICTURE3'].str.split(';', expand=True)
participants_pic.drop(['kalibracja_PICTURE1', 'kalibracja_PICTURE2', 'kalibracja_PICTURE3'], axis = 1, inplace = True)


#Changing column place "Dodatkowe uwagi" to last position
column_to_move = participants_pic.pop(participants_pic.columns[3])
participants_pic.insert(len(participants_pic.columns), column_to_move.name, column_to_move)

#
participants_pic['Dodatkowe uwagi'] = participants_pic['Dodatkowe uwagi'].notnull().astype(int)
participants_pic.head()

,Numer osoby,płeć,kolor oczu,kalibracja_PICTURE1_value_1,kalibracja_PICTURE1_value_2,kalibracja_PICTURE2_value_1,kalibracja_PICTURE2_value_2,kalibracja_PICTURE3_value_1,kalibracja_PICTURE3_value_2,Dodatkowe uwagi
0,S01,k,zielone,NaN,NaN,NaN,NaN,NaN,NaN,0
1,S02,k,NaN,3.096,0.215,2.819,0.220,3.015,0.220,0
2,S03,k,piwne,3.227,0.156,2.674,0.156,2.575,0.136,1
3,S04,k,piwne,3.137,0.222,2.129,0.204,2.924,0.207,0
4,S05,m,piwne,2.400,0.183,2.786,0.178,2.381,0.173,0


# Folder ratings - for patient S05
- each CSV file - details regarding specific participant assessment of picture:
    + image9 - image name from IAPS
    + Valence_rating.response - valence for each picture [1-9]
    + Arousal_rating.response - arousal for each picture [1-9]

Looking on CSV extension for S05 patient

In [5]:
#reading the file
rating_1 = pd.read_csv(r"C:\Users\48660\Desktop\Development Workshop\ratings\2022_Dec_22_0959S05.csv")
print(f"Shape of rating_1 before dropping NaN rows of image9: {rating_1.shape}")
#dropping rows without image9 provided (NaN)
rating_1.dropna(subset=['image9'], inplace = True)
print(f"Shape of rating_1 before after NaN rows of image9: {rating_1.shape}")

#looking for NaN values in columns (if sum(NaNs) >= 240 then delete column)
columns_dropped = []

print("Amount of NaN values in columns: ")
for column in rating_1.columns:
    nan_sum = rating_1[column].isna().sum()
    print(f"Column: --- {column} --- amount of NaN values: --- {nan_sum} ---")
    if nan_sum >= 240:
        rating_1.drop([column], axis = 1, inplace = True)
        columns_dropped.append(column)

print(f"Amount of columns dropped: {len(columns_dropped)}")

rating_1.head()

Shape of rating_1 before dropping NaN rows of image9: (249, 63)
Shape of rating_1 before after NaN rows of image9: (245, 63)
Amount of NaN values in columns: 
Column: --- image9 --- amount of NaN values: --- 0 ---
Column: --- trials.thisRepN --- amount of NaN values: --- 0 ---
Column: --- trials.thisTrialN --- amount of NaN values: --- 0 ---
Column: --- trials.thisN --- amount of NaN values: --- 0 ---
Column: --- trials.thisIndex --- amount of NaN values: --- 0 ---
Column: --- trials_2.thisRepN --- amount of NaN values: --- 240 ---
Column: --- trials_2.thisTrialN --- amount of NaN values: --- 240 ---
Column: --- trials_2.thisN --- amount of NaN values: --- 240 ---
Column: --- trials_2.thisIndex --- amount of NaN values: --- 240 ---
Column: --- text_2.started --- amount of NaN values: --- 245 ---
Column: --- text_2.stopped --- amount of NaN values: --- 245 ---
Column: --- text_9.started --- amount of NaN values: --- 245 ---
Column: --- text_9.stopped --- amount of NaN values: --- 245 --

,image9,trials.thisRepN,trials.thisTrialN,trials.thisN,trials.thisIndex,text_4.started,image.started,mouse.started,Valence.started,Valence_rating.response,...,Valence_rating.started,Arousal.started,Arousal_rating.response,Arousal_rating.rt,Arousal_rating.started,Kod_osoby_badanej,date,expName,psychopyVersion,frameRate
3,hi_pos_IAPS_4658.png,0.0,0.0,0.0,9.0,101.611497,101.611497,0.006476,112.247960,8.0,...,0.008486,112.247960,7.0,28.440,0.008486,S05,2022_Dec_22_0959,rating,2021.1.4,59.995776
4,hi_neg_IAPS_9584.png,0.0,1.0,1.0,51.0,140.734119,140.734119,0.011315,144.187693,3.0,...,0.014416,144.187693,5.0,23.562,0.014416,S05,2022_Dec_22_0959,rating,2021.1.4,59.995776
5,pos_IAPS_7350.png,0.0,2.0,2.0,14.0,167.800066,167.800066,0.014229,170.037210,7.0,...,0.001760,170.037210,3.0,8.023,0.001760,S05,2022_Dec_22_0959,rating,2021.1.4,59.995776
6,hi_pos_NAPS_Objects_097_h.png,0.0,3.0,3.0,62.0,178.115556,178.115556,0.014285,182.658574,8.0,...,0.002057,182.658574,1.0,3.471,0.002057,S05,2022_Dec_22_0959,rating,2021.1.4,59.995776
7,neg_NAPS_People_142_h.png,0.0,4.0,4.0,96.0,189.973604,189.973604,0.011526,201.342733,4.0,...,0.012597,201.342733,5.0,2.532,0.012597,S05,2022_Dec_22_0959,rating,2021.1.4,59.995776


Rating folder ---- xlsx file ---- probably not necessarry for our analysis

In [25]:
rating_2 = pd.read_excel(r"C:\Users\48660\Desktop\Development Workshop\ratings\2022_Dec_22_0959S05.xlsx")
rating_2.head()


,n,key_resp_6.keys_raw,key_resp_6.rt_mean,key_resp_6.rt_raw,key_resp_6.rt_std,key_resp_6.started_mean,key_resp_6.started_raw,key_resp_6.started_std,key_resp_6.stopped_raw,text.started_mean,text.started_raw,text.started_std,text.stopped_raw,order
0,1,'space',2.154411,2.154411,0.0,619.075073,619.075073,0.0,NaN,619.075073,619.075073,0.0,NaN,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,extraInfo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kod_osoby_badanej,S05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,date,2022_Dec_22_0959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# folder physiology_bitalino/bitalino_pic - 
+ one or several CSV files (depending if there was a problem during experiment) per participant
+ reach CSV file - raw measurement regarding specific modality:
    + lp - just a natural number starting from 0 and increased by 1 with each raw of data
    + EKG - ECG (electrocardiography) value
    + light - lightning measurement value
    + EDA - EDA (elctrodermal activity) value
    + HR - HR (heart rate) value
    + trigger - trigger into - what is presented to a participant in a moment (trigger corresponds to IAPS name; trigger 10000 means a 5 seconds break between the pictures; there is no baseline for physiology recordered here)

In [7]:
bitalino = pd.read_csv(r"C:\Users\48660\Desktop\Development Workshop\bitalino\S04_pic2_BITAL.csv")
bitalino.head()

,lp,EKG,light,blank,EDA,HR,trigger
0,0,31292,1164,0,9364,inf,0
1,1,31160,1352,0,9386,inf,0
2,2,31084,1200,0,9344,inf,0
3,3,31056,1125,0,9345,inf,0
4,4,31036,1302,0,9364,inf,0


# folder pupillabs_exported/pupillabs_pic/

ANNOTATIONS - one or several CSV files (depending if there was a problem during experiment) per participant:
+ each CSV file - triggers and corresponding timestamps
    + timestamp - a moment in time since the begining of procedure when a trigger was presented
    + trigger - trigger into - what is presented to a participant in a moment (trigger corresponds to IAPS name; trigger 10000 means a 5 seconds break between the pictures)

In [20]:
annotations = pd.read_csv(r"C:\Users\48660\Desktop\Development Workshop\pupillabs_exported - pics\annotations\2022_12_22_09-27 S05_pic_000_a.csv")
annotations.head()
#annotations.info()

,index,timestamp,label,duration,UnityTriggertrigger
0,123,559.858582,UnityTrigger,0.0,42131
1,274,564.861237,UnityTrigger,0.0,10000
2,415,569.587952,UnityTrigger,0.0,40422
3,566,574.595612,UnityTrigger,0.0,10000
4,696,578.939362,UnityTrigger,0.0,42032


# FIXATIONS
fixation of an eye on a specific location - three or several CSV files (depending if there was a problem during experiment) per participant:
- each CSV file - timestamps and positions
    + start_timestamp - time when the fixation started
    + duration - how long was the fixation
    + dispersion - dispersion of individual recorded eye positions that were considered to belong to a single fixation

In [13]:
fixations = pd.read_csv(r"C:\Users\48660\Desktop\Development Workshop\pupillabs_exported - pics\fixations\2022_12_22_09-27 S05_pic_000_f.csv")
fixations.info()


#calculating the mean of base_data column
def calculate_mean(row):
    numbers = [float(x) for x in row.split()]
    mean = np.mean(numbers)
    return mean
fixations['base_data_mean'] = fixations['base_data'].apply(calculate_mean)
fixations.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 941 non-null    int64  
 1   start_timestamp    941 non-null    float64
 2   duration           941 non-null    float64
 3   start_frame_index  941 non-null    int64  
 4   end_frame_index    941 non-null    int64  
 5   norm_pos_x         941 non-null    float64
 6   norm_pos_y         941 non-null    float64
 7   dispersion         941 non-null    float64
 8   confidence         941 non-null    float64
 9   method             941 non-null    object 
 10  gaze_point_3d_x    941 non-null    float64
 11  gaze_point_3d_y    941 non-null    float64
 12  gaze_point_3d_z    941 non-null    float64
 13  base_data          941 non-null    object 
dtypes: float64(9), int64(3), object(2)
memory usage: 103.1+ KB


,id,start_timestamp,duration,start_frame_index,end_frame_index,norm_pos_x,norm_pos_y,dispersion,confidence,method,gaze_point_3d_x,gaze_point_3d_y,gaze_point_3d_z,base_data,base_data_mean
0,0,555.656295,103.9380,0,1,0.083601,0.633259,1.354630,0.985735,3d gaze,-225.072840,-40.518780,422.294283,555.6562949999998 555.6642489999995 555.672273...,555.708355
1,1,556.780170,122.0860,32,35,0.520707,0.369121,1.471917,0.792438,3d gaze,8.222890,30.700249,327.864107,556.7801704999997 556.7841399999993 556.788161...,556.839897
2,2,557.372858,195.3595,49,55,0.510774,0.418027,1.501905,0.908244,3d gaze,54.033054,391.824746,6436.362621,557.3728579999997 557.3754449999997 557.376830...,557.467440
3,3,558.144141,119.9930,73,76,0.525956,0.464647,1.444530,0.923740,3d gaze,323.077978,281.744417,9973.936067,558.1441414999995 558.1481494999994 558.152147...,558.204812
4,4,558.268170,151.9650,76,81,0.522615,0.429604,1.418351,0.911343,3d gaze,136.763532,249.249462,5181.788202,558.2681704999995 558.2723574999995 558.276534...,558.344137


# pupil_positions 
- each CSV file - timestamp and pupil diameters (doubled rows since there are two metods of detection: 2d c++, pye3d 0.3.0 real-time)
    + pupil_timestamp - time of detection
    + eye_id - left or right
    + diameter - pupil diameter

In [21]:
positions = pd.read_csv(r"C:\Users\48660\Desktop\Development Workshop\pupillabs_exported - pics\positions\2022_12_22_09-19 S05_pic_000_p.csv")
#due to model_id has 0 good samples we delete this column
positions.drop(['model_id'], axis = 1, inplace = True)
positions.info()
#half of the columns have positions.shape[0] / 2 non null values ????
positions.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198396 entries, 0 to 198395
Data columns (total 33 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   pupil_timestamp            198396 non-null  float64
 1   world_index                198396 non-null  int64  
 2   eye_id                     198396 non-null  int64  
 3   confidence                 198396 non-null  float64
 4   norm_pos_x                 198396 non-null  float64
 5   norm_pos_y                 198396 non-null  float64
 6   diameter                   198396 non-null  float64
 7   method                     198396 non-null  object 
 8   ellipse_center_x           198396 non-null  float64
 9   ellipse_center_y           198396 non-null  float64
 10  ellipse_axis_a             198396 non-null  float64
 11  ellipse_axis_b             198396 non-null  float64
 12  ellipse_angle              198396 non-null  float64
 13  diameter_3d                99

,pupil_timestamp,world_index,eye_id,confidence,norm_pos_x,norm_pos_y,diameter,method,ellipse_center_x,ellipse_center_y,...,circle_3d_normal_y,circle_3d_normal_z,circle_3d_radius,theta,phi,projected_sphere_center_x,projected_sphere_center_y,projected_sphere_axis_a,projected_sphere_axis_b,projected_sphere_angle
0,86.621965,0,1,0.758826,0.536952,0.466931,18.161634,2d c++,103.094780,102.349174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,86.621965,0,1,0.758826,0.537174,0.467157,18.168477,pye3d 0.3.0 real-time,103.137477,102.305767,...,0.413694,-0.874244,0.523304,1.144289,-1.853631,121.864263,59.233975,202.378702,202.378702,0.0
2,86.624622,0,0,0.733817,0.610242,0.551886,18.627298,pye3d 0.3.0 real-time,117.166491,86.037938,...,-0.487426,-0.857523,1.371671,2.079936,-1.381236,109.054367,110.213066,106.064161,106.064161,0.0
3,86.624622,0,0,0.733817,0.610226,0.551965,18.618183,2d c++,117.163414,86.022686,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,86.626173,0,1,0.762146,0.537365,0.467062,17.916333,pye3d 0.3.0 real-time,103.173988,102.324127,...,0.413759,-0.874264,0.517500,1.144217,-1.853439,121.864263,59.233975,202.378702,202.378702,0.0
5,86.626173,0,1,0.762146,0.537390,0.466660,17.916977,2d c++,103.178963,102.401218,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,86.632513,0,1,0.331012,0.354824,0.664662,21.359143,pye3d 0.3.0 real-time,68.126294,64.384897,...,0.282140,-0.858951,0.573879,1.284772,-2.032431,121.864263,59.233975,202.378702,202.378702,0.0
7,86.632513,0,1,0.684912,0.536971,0.467379,17.917400,2d c++,103.098526,102.263325,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,86.632515,0,0,0.865093,0.609858,0.550351,18.534626,2d c++,117.092659,86.332668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,86.632515,0,0,0.865093,0.609912,0.550371,18.540328,pye3d 0.3.0 real-time,117.103111,86.328755,...,-0.485353,-0.857556,1.363933,2.077564,-1.374665,109.054367,110.213066,106.064161,106.064161,0.0
